In [22]:
import re
import pandas as pd
from nltk.corpus import stopwords

In [23]:
# Ref.: https://stackoverflow.com/questions/5552555/unicodedecodeerror-invalid-continuation-byte
df = pd.read_csv("rt_reviews.csv", encoding="latin-1")

In [24]:
df.head()

,Freshness,Review
0,fresh,"Manakamana doesn't answer any questions, yet ..."
1,fresh,Wilfully offensive and powered by a chest-thu...
2,rotten,It would be difficult to imagine material mor...
3,rotten,Despite the gusto its star brings to the role...
4,rotten,If there was a good idea at the core of this ...


### Preprocessing the Review

In [25]:
# Stripping leading and trailing empty space characters
# Converting the text to lower case letters
df["preprocessed_review"] = df["Review"].apply(lambda x: x.strip().lower())

In [27]:
# # Ref.: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
# contractions = { 
#     "ain't": "am not",
#     "aren't": "are not",
#     "can't": "cannot",
#     "can't've": "cannot have",
#     "'cause": "because",
#     "could've": "could have",
#     "couldn't": "could not",
#     "couldn't've": "could not have",
#     "didn't": "did not",
#     "doesn't": "does not",
#     "don't": "do not",
#     "hadn't": "had not",
#     "hadn't've": "had not have",
#     "hasn't": "has not",
#     "haven't": "have not",
#     "he'd": "he had",
#     "he'd've": "he would have",
#     "he'll": "he will",
#     "he'll've": "he will have",
#     "he's": "he has",
#     "how'd": "how did",
#     "how'd'y": "how do you",
#     "how'll": "how will",
#     "how's": "how has",
#     "I'd": "I had",
#     "I'd've": "I would have",
#     "I'll": "I will",
#     "I'll've": "I will have",
#     "I'm": "I am",
#     "I've": "I have",
#     "isn't": "is not",
#     "it'd": "it would",
#     "it'd've": "it would have",
#     "it'll": "it will",
#     "it'll've": "it will have",
#     "it's": "it is",
#     "let's": "let us",
#     "ma'am": "madam",
#     "mayn't": "may not",
#     "might've": "might have",
#     "mightn't": "might not",
#     "mightn't've": "might not have",
#     "must've": "must have",
#     "mustn't": "must not",
#     "mustn't've": "must not have",
#     "needn't": "need not",
#     "needn't've": "need not have",
#     "o'clock": "of the clock",
#     "oughtn't": "ought not",
#     "oughtn't've": "ought not have",
#     "shan't": "shall not",
#     "sha'n't": "shall not",
#     "shan't've": "shall not have",
#     "she'd": "she would",
#     "she'd've": "she would have",
#     "she'll": "she will",
#     "she'll've": "she will have",
#     "she's": "she is",
#     "should've": "should have",
#     "shouldn't": "should not",
#     "shouldn't've": "should not have",
#     "so've": "so have",
#     "so's": "so is",
#     "that'd": "that would",
#     "that'd've": "that would have",
#     "that's": "that is",
#     "there'd": "there would",
#     "there'd've": "there would have",
#     "there's": "there is",
#     "they'd": "they had",
#     "they'd've": "they would have",
#     "they'll": "they will",
#     "they'll've": "they will have",
#     "they're": "they are",
#     "they've": "they have",
#     "to've": "to have",
#     "wasn't": "was not",
#     "we'd": "we would",
#     "we'd've": "we would have",
#     "we'll": "we will",
#     "we'll've": "we will have",
#     "we're": "we are",
#     "we've": "we have",
#     "weren't": "were not",
#     "what'll": "what will",
#     "what'll've": "what will have",
#     "what're": "what are",
#     "what's": "what is",
#     "what've": "what have",
#     "when's": "when is",
#     "when've": "when have",
#     "where'd": "where did",
#     "where's": "where is",
#     "where've": "where have",
#     "who'll": "who will",
#     "who'll've": "who will have",
#     "who's": "who is",
#     "who've": "who have",
#     "why's": "why is",
#     "why've": "why have",
#     "will've": "will have",
#     "won't": "will not",
#     "won't've": "will not have",
#     "would've": "would have",
#     "wouldn't": "would not",
#     "wouldn't've": "would not have",
#     "y'all": "you all",
#     "y'all'd": "you all would",
#     "y'all'd've": "you all would have",
#     "y'all're": "you all are",
#     "y'all've": "you all have",
#     "you'd": "you would",
#     "you'd've": "you would have",
#     "you'll": "you will",
#     "you'll've": "you will have",
#     "you're": "you are",
#     "you've": "you have"}

In [28]:
stop_words = set(stopwords.words('english'))

In [29]:
df["preprocessed_review"] = df["preprocessed_review"].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))

In [30]:
df["preprocessed_review"]

0         manakamana answer questions, yet makes point: ...
1         wilfully offensive powered chest-thumping mach...
2         would difficult imagine material wrong spade l...
3         despite gusto star brings role, hard ride shot...
4         good idea core film, buried unsightly pile fla...
                                ...                        
479995    zemeckis seems unable admit motion capture ani...
479996    movies like kids right -- beautifully written,...
479997    film-savvy audiences soon catch onto winterbot...
479998                              odd yet enjoyable film.
479999    animation studio, even beloved pixar, quite re...
Name: preprocessed_review, Length: 480000, dtype: object

In [31]:
# Using regluar expression to select only alphabet characters into the Dataset

df["preprocessed_review"] = df["preprocessed_review"].apply(lambda x: " ".join(re.findall(r'[a-z]+', x)))

In [32]:
df["preprocessed_review"]

0         manakamana answer questions yet makes point ne...
1         wilfully offensive powered chest thumping mach...
2         would difficult imagine material wrong spade l...
3         despite gusto star brings role hard ride shotg...
4         good idea core film buried unsightly pile flat...
                                ...                        
479995    zemeckis seems unable admit motion capture ani...
479996    movies like kids right beautifully written imp...
479997    film savvy audiences soon catch onto winterbot...
479998                               odd yet enjoyable film
479999    animation studio even beloved pixar quite repl...
Name: preprocessed_review, Length: 480000, dtype: object

In [33]:
df.head()

,Freshness,Review,preprocessed_review
0,fresh,"Manakamana doesn't answer any questions, yet ...",manakamana answer questions yet makes point ne...
1,fresh,Wilfully offensive and powered by a chest-thu...,wilfully offensive powered chest thumping mach...
2,rotten,It would be difficult to imagine material mor...,would difficult imagine material wrong spade l...
3,rotten,Despite the gusto its star brings to the role...,despite gusto star brings role hard ride shotg...
4,rotten,If there was a good idea at the core of this ...,good idea core film buried unsightly pile flat...


In [38]:
total_samples = len(df)

class_counts = dict(df["Freshness"].value_counts())

total_samples, class_counts

(480000, {'fresh': 240000, 'rotten': 240000})

In [40]:
class_probabilities = {k:v/total_samples for k, v in class_counts.items()}

In [41]:
class_probabilities

{'fresh': 0.5, 'rotten': 0.5}

In [44]:
from tqdm import tqdm

unique_words = [df.loc[i, "preprocessed_review"].split() for i in range(len(df))]

In [74]:
fresh_class_words = {}
rotten_class_words = {}

fresh_word_count = 0
rotten_word_count = 0

unique_words = []


for i in tqdm(range(10000)):
    review = df.loc[i, "preprocessed_review"]
    
    if df.loc[i, "Freshness"]=="fresh":
        for word in review.split():
            if word not in fresh_class_words.keys():
                fresh_class_words[word] = 1
            else:
                fresh_class_words[word] += 1
            
            if word not in unique_words:
                unique_words.append(word)
                
            fresh_word_count += 1
    else:
        for word in review.split():
            if word not in rotten_class_words.keys():
                rotten_class_words[word] = 1
            else:
                rotten_class_words[word] += 1
            
            if word not in unique_words:
                unique_words.append(word)
                
            rotten_word_count += 1

100%|██████████| 10000/10000 [00:13<00:00, 733.82it/s]


In [75]:
fresh_class_words = {k:(v+1/(fresh_word_count + len(unique_words))) for k, v in fresh_class_words.items()}

In [76]:
rotten_class_words = {k:(v+1/(rotten_word_count + len(unique_words))) for k, v in rotten_class_words.items()}

In [77]:
review = df.loc[10, "preprocessed_review"]

In [78]:
review

'bell s debut feature addresses subject humor intelligence approaching issue performance enhancement every conceivable angle'

In [79]:
import math

fresh_prob = math.log(class_probabilities["fresh"])

rotten_prob = math.log(class_probabilities["rotten"])

In [81]:
for word in review.split():
    if word in fresh_class_words:
        fresh_prob += math.log(fresh_class_words[word])
    else:
        fresh_prob += math.log(1/(fresh_word_count + len(unique_words)))
        
    if word in rotten_class_words:
        rotten_prob += math.log(rotten_class_words[word])
    else:
        rotten_prob += math.log(1/(rotten_word_count + len(unique_words)))

In [82]:
fresh_prob, rotten_prob

(85.14628375923954, 13.806625558470662)

In [83]:
df.loc[10]

Freshness                                                          fresh
Review                  Bell's debut feature addresses its subject wi...
preprocessed_review    bell s debut feature addresses subject humor i...
Name: 10, dtype: object